In [1]:
import json
import sys
import gc
import os
import uproot

# Añade el directorio padre al sys.path
sys.path.append(os.path.abspath("/eos/home-d/dcostasr/SWAN_projects/2025_data"))
sys.path.append('/eos/home-d/dcostasr/SWAN_projects/software/hipy')

import numpy       as np
import pandas      as pd
import hipy.pltext as pltext
import awkward     as ak
import matplotlib.pyplot as plt

from wcte.brbtools import sort_run_files, get_part_files, select_good_parts
from tqdm          import tqdm

%load_ext autoreload
%autoreload 2

pltext.style()

In [6]:
def get_files_from_part(part_file, run_files):
    """
    Input a part_file number and the run_files list created with sort_run_files.
    Extracts all the card_id, channel_id, charge and time information from the hits.
    Returns the arrays for the variables for the part_file.
    """
    tree = uproot.open(run_files[part_file]+":WCTEReadoutWindows")
    file_hit_card_ids    = tree["hit_mpmt_card_ids"].array()
    file_hit_channel_ids = tree["hit_pmt_channel_ids"].array()
    file_hit_charges     = tree["hit_pmt_charges"].array()
    file_hit_times       = tree["hit_pmt_times"].array()
    file_event_number    = tree["event_number"].array()

    return file_hit_card_ids, file_hit_channel_ids, file_hit_charges, file_hit_times, file_event_number

In [3]:
run        = 1626
run_files  = sort_run_files(f"/eos/experiment/wcte/data/2025_commissioning/offline_data/{run}/WCTE_offline_R{run}S*P*.root")
part_files = get_part_files(run_files)
good_parts = select_good_parts(part_files, run_files)

Selecting Good Parts: 100%|██████████| 9/9 [00:00<00:00, 47.12it/s]


In [4]:
with open('/eos/home-d/dcostasr/SWAN_projects/NiCf/offline_trigger/mmc_map_R1609.json') as f:
    mcc_map = json.load(f)

d = {}
for k,v in zip(mcc_map.keys(), mcc_map.values()):
    card, channel = [int(i) for i in str(int(k)/100).split(".")]
    d[(card, channel)] = v

mcc_map = d

In [8]:
events = np.array([])
times  = np.array([])
cards  = np.array([])
channels = np.array([])

evt_offset = 0
for ipar in tqdm(good_parts[:2], total=len(good_parts[:2])):
    files = get_files_from_part(good_parts[ipar], run_files)

    events   = np.concatenate((events, np.array(files[4])), axis=None)
    times    = np.concatenate((times,  np.array(files[3])), axis=None)
    cards    = np.concatenate((cards,  np.array(files[0])), axis=None)
    channels = np.concatenate((channels,  np.array(files[1])), axis=None)
        
    # df    = create_df_from_file_fast(files, mcc_map)
    # df['evt'] += evt_offset
    # evt_offset = df['evt'].max() + 1
    # dfs.append(df)

  0%|          | 0/2 [00:01<?, ?it/s]


ValueError: cannot convert to RegularArray because subarray lengths are not regular (in compiled code: https://github.com/scikit-hep/awkward/blob/awkward-cpp-45/awkward-cpp/src/cpu-kernels/awkward_ListOffsetArray_toRegularArray.cpp#L22)